# Retrieve Catalog Data from `catalogue.json`

The file `catalogue.json` contains all the catalog data for an [ORACC](http://oracc.org) project (for general information, see the [Oracc Open Data](http://oracc.org/doc/opendata) page). The `zip` that contains all JSON files of a particular project can be found at `http://build-oracc.museum.upenn.edu/json/[PROJECT].zip`. In the URL replace [PROJECT] with your project name (e.g. `dcclt`). For sub-projects the URL pattern is `http://build-oracc.museum.upenn.edu/json/[PROJECT]-[SUBPROJECT].zip`. 

The main node in a `catalogue.json` file is called `members`. This node contains the information of all the fields and all the entries in the project catalog. This information is put in a Pandas DataFrame.

In [ ]:
import pandas as pd
import zipfile
import json
import requests
import errno
import os
import tqdm

## 0 Create Directories, if Necessary
The two directories needed for this script are `jsonzip` and `output`. If they do not exist they are created, else: do nothing.

For the code, see [Stack Overflow](http://stackoverflow.com/questions/18973418/os-mkdirpath-returns-oserror-when-directory-does-not-exist).

In [ ]:
directories = ['jsonzip', 'output']
for d in directories:
    try:
        os.mkdir(d)
    except OSError as exc:
        if exc.errno !=errno.EEXIST:
            raise
        pass

## 1.1 Input Project Names
We can download and manipulate multiple [ORACC](http://oracc.org) `zip` files at the same time. Note, however that different [ORACC](http://oracc.org) projects use different fields in their catalogs; not all catalogs are mutually compatible.

Provide project abbreviations, separated by a comma. Note that subprojects must be processed separately, they are not included in the main project. A subproject is named `[PROJECT]/[SUBPROJECT]`, for instance `saao/saa01`.

In [ ]:
projects = input('Project(s): ').lower().strip() # lowercase user input and strip accidental spaces

## 1.2 Split the List of Projects
Split the list of projects and create a list of project names.

In [ ]:
p = projects.split(',')               # split at each comma and make a list called `p`
p = [x.strip() for x in p]        # strip spaces left and right of each entry in `p`

## 1.2 Download the ZIP files
Download all the `json` files from `http://build-oracc.museum.upenn.edu/json/`. The file is called `[PROJECT].zip` (for instance: `dcclt.zip`). For subprojects the file is called `[PROJECT-SUBPROJECT].zip` (for instance `cams-gkab.zip`). 

For larger projects (such as [DCCLT](http://oracc.org/dcclt)) the `zip` file may be 25Mb or more. Downloading may take some time and it may be necessary to chunk the downloading process. The `iter_content()` function in the `requests` library takes care of that. For the chunking code see [this page](https://www.smallsurething.com/how-to-read-a-file-properly-in-python/).

If you have downloaded the files by hand (and put them in the `jsonzip` directory) you may skip this cell and jump directly to section 2.

In [ ]:
non_existent = []
CHUNK = 16 * 1024
for project in tqdm.tqdm(p):
    project = project.replace('/', '-')
    url = "http://build-oracc.museum.upenn.edu/json/" + project + ".zip"
    file = 'jsonzip/' + project + '.zip'
    r = requests.get(url)
    if r.status_code == 200:
        print("Downloading " + url + " saving as " + file)
        with open(file, 'wb') as f:
            for c in r.iter_content(chunk_size=CHUNK):
                f.write(c)
    else:
        print(url + " does not exist.")
        non_existent.append(project.replace('-', '/'))
p = [i for i in p if i not in non_existent] # remove non-existing project names from list

## 2 Extract Catalogue Data from `JSON` files
The code in this cell will iterate through the list of projects entered above (1.1). For each project the `JSON` zip file, named `[PROJECT].zip` is located in the directory `jsonzip`. Each of these `zip` files includes a file called `catalogue.json`. This file is extracted and read with the command `json.loads()`, which reads the json data and transforms it into a JSON object - a sequence of names and values.

The JSON object is transformed into a Pandas Dataframe. The dataframe needs to be transposed (`.T`), so that the P, Q, and X numbers become indexes or row names (rather than column names), and each column represents a field in the catalog.  The individual dataframes (one for each project requested) are concatenated. Since individual [ORACC](http://oracc.org) project catalogs may have different fields, the dataframes may have different column names. By default Pandas concatenation uses an `outer join` so that all column names of all the catalogs are preserved.

In [ ]:
df = pd.DataFrame() # create an empty dataframe
for project in p:
    file = "jsonzip/" + project.replace("/", "-") + ".zip"
    try:
        z = zipfile.ZipFile(file)       # create a Zipfile object
    except:
        print(file + " does not exist or is not a proper ZIP file")
        continue
    try:
        st = z.read(project + '/catalogue.json').decode('utf-8')  #read and decode the catalogue.json file of one project
                                                                # the result is a string object
    except:
        print(project + '/catalogue.json' + ' is not available or not complete')
        continue
    cat = json.loads(st)
    cat = cat['members']  # select the 'members' node 
    for item in cat.values():
        item["project"] = project # add project name as separate field
    cat_df = pd.DataFrame(cat).T
    df = pd.concat([df, cat_df], sort=True)  # sort=True is necessary in case catalogs have a different set of fields
df

## 3. Clean the Dataframe
The function `fillna('')` will put a blank (instead of `NaN`) in all fields that have no entry.

In [ ]:
df = df.fillna('')
df

## 4 Select Relevant Fields
[ORACC](http://oracc.org) catalogs may have custom fields, the only fields that are obligatory are `id_text` (the P, Q, or X number that identifies the text, for instance "P243546") and `designation` (the human-readable reference, for instance "VS 17, 012"). The example code below works with field names that are available in (almost) every catalog. Adjust the code to your data and your needs.

In [ ]:
df1 = df.loc[: , ['designation', 'period', 'provenience',
        'museum_no', 'project', 'id_text']]
df1

## 5.1 Save as CSV
Save the resulting data set as a `csv` file. `UTF-8` encoding is the encoding with the widest support in text analysis (and also the encoding used by [ORACC](http://oracc.org)). If you intend to use the catalog file in Excel, however, it is better to use `utf-16` encoding.

In [ ]:
filename = 'catalog.csv'
with open('output/' + filename, 'w', encoding='utf-8') as w:
    df1.to_csv(w, index=False)

## 5.2 Save with Pickle
One may pickle a file either with the `pickle` library or directly from within `Pandas` with the `to_pickle()` function. A pickled file preserves the data structure of the dataframe, which is an advantage over saving as `csv`. The pickle file is a binary file, so we must open the file with the `wb` (write binary) option and we cannot give an encoding. To open the pickled file one may use the `read_pickle()` function from the `pandas` library, as in

```python
with open('output/catalog.p', 'rb') as o: 
    df = pd.read_pickle(o)
```

In [ ]:
filename = "catalog.p"
with open('output/' + filename, 'wb') as w:
    df1.to_pickle(w)